# Questions to the data
1. Does the number of hauses in all houses groups remain stable or there were a construction side finished in the given time frame?
2. What was the weather in the region?
3. What was the price for gas/oil/elictricity?

In [92]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Tuple, Any
import time
from sklearn import linear_model
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklego.preprocessing import RepeatingBasisFunction
import warnings
warnings.filterwarnings("ignore")

In [93]:
data_original = pd.read_csv('../data/public_data/train.csv')
final_index = data_original['pseudo_id']

In [94]:
data_filtered = pd.read_csv('../data/train_filtered.csv')

In [95]:
# drop index for feature preparation
data_ = data_filtered.drop(columns='pseudo_id')
# convert dates to pandas datetime
data_.columns = [datetime.strptime(c, "%Y-%m-%d %H:%M:%S") for c in data_.columns]
data_.head()

,2017-01-01 00:00:00,2017-01-01 00:30:00,2017-01-01 01:00:00,2017-01-01 01:30:00,2017-01-01 02:00:00,2017-01-01 02:30:00,2017-01-01 03:00:00,2017-01-01 03:30:00,2017-01-01 04:00:00,2017-01-01 04:30:00,...,2019-08-28 19:00:00,2019-08-28 19:30:00,2019-08-28 20:00:00,2019-08-28 20:30:00,2019-08-28 21:00:00,2019-08-28 21:30:00,2019-08-28 22:00:00,2019-08-28 22:30:00,2019-08-28 23:00:00,2019-08-28 23:30:00
0,45.022999,39.985001,36.569500,34.748001,35.972000,38.438999,36.591000,36.315498,32.660500,31.564501,...,24.2880,23.994,26.1995,25.0270,23.0665,26.093,23.4295,25.4715,26.246,22.6020
1,2.931000,1.641000,2.260000,2.273000,2.651000,3.137000,2.532000,3.142000,2.528000,2.637000,...,2.5700,1.446,1.5230,1.5630,2.5880,2.190,1.4860,2.5270,2.288,1.7940
2,11.014000,12.652500,10.824001,13.748500,12.383000,12.342000,13.413000,11.484000,11.510500,11.480751,...,6.3565,5.766,5.4955,5.0885,6.8140,7.492,5.7705,6.8240,6.072,6.7205
3,55.813000,49.040001,49.095001,41.132999,45.660000,48.477001,50.539001,45.737000,42.680000,42.782501,...,32.6460,30.439,30.2470,31.2660,34.3390,33.076,33.1080,33.7260,30.009,34.8400
4,26.925001,28.118000,25.600000,28.091000,26.530001,23.858002,26.556000,27.714001,23.174002,23.545000,...,13.3980,13.280,13.7340,13.6060,14.7000,16.290,15.1240,15.3650,14.360,13.9350


In [96]:
freq = "H"
data_ = data_.T.resample(freq).sum()
data_.columns = [c+1 for c in range(len(data_.columns))]

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
2017-01-01 00:00:00,85.007999,4.572,23.666500,104.853001,55.043001,21.6700,8.517,50.174500,74.895000,22.242000,...,0.368,11.430000,9.319000,1.7450,5.8480,1.2320,3.041,6.9575,2.00650,6.69300
2017-01-01 01:00:00,71.317501,4.533,24.572501,90.228001,53.691000,20.8020,9.665,47.165501,63.504499,21.320000,...,0.680,11.414000,13.885499,1.8070,8.8230,0.6690,2.868,5.6150,1.41000,5.69800
2017-01-01 02:00:00,74.410999,5.788,24.725000,94.137001,50.388002,20.3090,10.970,51.724499,62.577000,23.941999,...,1.400,14.907000,9.272000,1.6500,8.5350,0.6450,2.892,6.6505,1.50150,5.95300
2017-01-01 03:00:00,72.906498,5.674,24.897000,96.276001,54.270000,19.0990,9.228,49.303999,64.942497,21.884999,...,0.803,12.920000,8.203000,1.3620,5.7210,1.4180,3.190,7.4195,1.25000,6.85600
2017-01-01 04:00:00,64.225000,5.165,22.991251,85.462502,46.719002,16.8785,7.688,23.951000,54.670252,17.892501,...,0.485,12.322999,8.815750,1.6685,6.2715,0.6275,1.803,7.2185,1.64475,4.22625


In [98]:
# check wether datetime in ascending order , it is important for time series
print(data_.index.is_monotonic)

True


In [99]:
data_[(data_.index >='2017-02-08 00:00:00') & (data_.index <= '2017-02-14 23:00:00')]

,1,2,3,4,5,6,7,8,9,10,...,52,53,54,55,56,57,58,59,60,61
2017-02-08 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-08 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-08 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-08 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-08 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-14 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-14 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-14 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
# Set dates for development phase
new_date_range = pd.date_range(start="2019-04-01", end="2019-09-05", freq="H")
data_ = data_.reindex(new_date_range)

In [102]:
data_=data_.drop(data_.index[-1])

In [103]:
# saving indexes for next steps
idx_test_date = data_.index[(data_[1] == 0) | (data_[1].isna())]

In [104]:
data_ = data_.fillna(method="ffill")

In [105]:
data_[data_.select_dtypes(np.float64).columns] = data_.select_dtypes(np.float64).astype(np.float32)

In [106]:
#plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':160})
#data_[1].plot()
#plt.title('Energy use forecasts for houshold group' + " 1")
#plt.show()

In [107]:
data_\
    .isna()\
    .sum()\
    .to_frame()\
    .assign(perc = lambda row: 100 * row[0] / data_.shape[0])\
    .rename(columns={0: 'Number of missed data  points', 'perc': '% of missed data points'})

,Number of missed data points,% of missed data points
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
5,0,0.0
...,...,...
57,0,0.0
58,0,0.0
59,0,0.0
60,0,0.0


# Adding features

In [108]:
def day_part(hour):
    part_of_day = 0
    if 0 <= hour <= 6:
        part_of_day = 1
    elif 6 < hour <= 12:
        part_of_day = 2
    elif 12 < hour <= 18:
        part_of_day = 3
    elif 18 < hour <= 23:
        part_of_day = 4
    return part_of_day

In [109]:
df = data_.copy(deep=True)

df["weekday"] = data_.index.weekday
df["dayofyear"] = data_.index.dayofyear
df["is_weekend"] = data_.index.weekday.isin([5, 6]).astype(np.int32)
df["month"] = data_.index.month
df["season"] = (data_.index.month % 12 + 3) // 3
df['Hour'] = df.index.hour
df['type_of_the_day'] = df.apply(lambda row : day_part(row['Hour']), axis = 1)

In [110]:
df_month = pd.get_dummies(df["month"], drop_first=True, prefix="month")
df_season = pd.get_dummies(df["season"], drop_first=True, prefix="season")
df_weekday = pd.get_dummies(df["weekday"], drop_first=True, prefix="weekday")
df_day_part = pd.get_dummies(df["type_of_the_day"], drop_first=True, prefix="day_part")
df = pd.concat([df, df_month, df_season, df_weekday,df_day_part], axis=1)
df.drop(columns=['month','season','weekday','type_of_the_day'], inplace=True)

In [115]:
def create_features(houshold_id,df=df) -> pd.DataFrame:
    df_new = df[[houshold_id, 'is_weekend',
                 #'month_2', 'month_3', 'month_4',
                 'month_5', 'month_6', 'month_7','month_8','month_9',
                 #'month_10','month_11','month_12',
                 #'season_2',
                 'season_3','season_4','weekday_1', 'weekday_2', 'weekday_3','weekday_4','weekday_5','weekday_6',
                 'day_part_2','day_part_3','day_part_4'
                 #'0_day_month','1_day_month','2_day_month','3_day_month','4_day_month','5_day_month','6_day_month','7_day_month','8_day_month','9_day_month','10_day_month','11_day_month'
                 ]]
    df_new['std'] = df_new[houshold_id].rolling(12).std().fillna(method="backfill")
    df_new['mean'] = df_new[houshold_id].rolling(12).mean().fillna(method="backfill")
    df_new['std'] = df_new[houshold_id].rolling(24).std().fillna(method="backfill")
    df_new['mean'] = df_new[houshold_id].rolling(24).mean().fillna(method="backfill")
    df_new['lag_1'] = df[houshold_id].shift(1).fillna(method="backfill")
    df_new['lag_2'] = df[houshold_id].shift(2).fillna(method="backfill")
    df_new['lag_3'] = df_new[houshold_id].shift(3).fillna(method="backfill")
    df_new['lag_4'] = df_new[houshold_id].shift(4).fillna(method="backfill")
    df_new['lag_5'] = df_new[houshold_id].shift(5).fillna(method="backfill")
    df_new['lag_6'] = df_new[houshold_id].shift(6).fillna(method="backfill")
    df_new['lag_7'] = df_new[houshold_id].shift(7).fillna(method="backfill")
    df_new['lag_8'] = df_new[houshold_id].shift(8).fillna(method="backfill")
    df_new['lag_9'] = df_new[houshold_id].shift(9).fillna(method="backfill")
    df_new['lag_10'] = df_new[houshold_id].shift(10).fillna(method="backfill")
    df_new['lag_11'] = df_new[houshold_id].shift(11).fillna(method="backfill")
    df_new['lag_12'] = df_new[houshold_id].shift(12).fillna(method="backfill")
    df_new['lag_13'] = df_new[houshold_id].shift(13).fillna(method="backfill")
    df_new['lag_14'] = df_new[houshold_id].shift(14).fillna(method="backfill")
    df_new['lag_15'] = df_new[houshold_id].shift(15).fillna(method="backfill")
    df_new['lag_16'] = df_new[houshold_id].shift(16).fillna(method="backfill")
    df_new['lag_17'] = df_new[houshold_id].shift(17).fillna(method="backfill")
    df_new['lag_18'] = df_new[houshold_id].shift(18).fillna(method="backfill")
    return df_new

In [116]:
def get_weeks(idx)-> List[List[pd._libs.tslibs.timestamps.Timestamp]]:
    idx = list(idx)
    count = 0
    hours_to_predict = []
    hours = []
    for i in range(len(idx)):
        if count > 167:
            hours_to_predict.append(hours)
            hours = []
            count = 0
        hours.append(idx[i])
        count += 1
    return hours_to_predict

In [117]:
def get_weeks_2(idx)-> List[List[pd._libs.tslibs.timestamps.Timestamp]]:
    idx = list(idx)
    weeks_to_predict = []
    week = []
    for i in range(1,len(idx)+1,1):
        #print(idx[i-1])
        if i%168 == 0:
            week.append(idx[i-1])
            weeks_to_predict.append(week)
            #print(week)
            week = []
        elif i%168 > 0:
            week.append(idx[i-1])
    return weeks_to_predict

In [118]:
hours_test_date = get_weeks_2(idx_test_date)

In [120]:
def data_split_accumlated(result, weeks, n, houshold) -> Tuple[Any, Any]:
    print(weeks[n])
    result_splitted = result[result.index < weeks[n][0]]
    result_splitted_features = result_splitted.drop([houshold], axis=1)
    result_splitted_target = result_splitted[houshold]
    #print("target: ", result_splitted_target)
    result_splitted_to_predict = result[(result.index >= weeks[n][0]) & (result.index <= weeks[n][167])]
    result_splitted_to_predict_features = result_splitted_to_predict.drop([houshold], axis=1)
    result_splitted_to_predict_target = result_splitted_to_predict[houshold]
    result_splitted_to_predict_target = result_splitted_to_predict_target.to_frame()

    return result_splitted_features, result_splitted_target, result_splitted_to_predict_features, result_splitted_to_predict_target

In [122]:
"""all_predictions = []


start_time = time.time()

for houshold in data_.columns:
    houshold_predictions = []

    print("***** Houshold " + str(houshold) + " dataset created ****** ")
    for week in range(len(hours_test_date)):
        #print("week "+  str(week) + " splitting started")
        result = create_features(houshold)
        features, target, features_predict, target_predict = data_split_accumlated(result, hours_test_date, week, houshold)

        model_linear = LinearRegression()

        model_linear.fit(features, target)

        print("trained on " + str(week))

        prediction = model_linear.predict(features_predict)

        houshold_predictions.append(prediction)

        week_timestamps = hours_test_date[week]

        target_predict['predict'] = prediction

        df.loc[week_timestamps[0]:week_timestamps[6],houshold] = target_predict['predict']

        print("=============================================")
        print(prediction)
        #print(df.loc[week_timestamps[0]:week_timestamps[6],houshold])
        print("=============================================")

    all_predictions.append(houshold_predictions)

print("--- %s seconds ---" % (time.time() - start_time))
"""

'all_predictions = []\n\n\nstart_time = time.time()\n\nfor houshold in data_.columns:\n    houshold_predictions = []\n\n    print("***** Houshold " + str(houshold) + " dataset created ****** ")\n    for week in range(len(hours_test_date)):\n        #print("week "+  str(week) + " splitting started")\n        result = create_features(houshold)\n        features, target, features_predict, target_predict = data_split_accumlated(result, hours_test_date, week, houshold)\n\n        model_linear = LinearRegression()\n\n        model_linear.fit(features, target)\n\n        print("trained on " + str(week))\n\n        prediction = model_linear.predict(features_predict)\n\n        houshold_predictions.append(prediction)\n\n        week_timestamps = hours_test_date[week]\n\n        target_predict[\'predict\'] = prediction\n\n        df.loc[week_timestamps[0]:week_timestamps[6],houshold] = target_predict[\'predict\']\n\n        print("=============================================")\n        print(pr

In [123]:
all_predictions = []

start_time = time.time()

for houshold in data_.columns:

    houshold_predictions = []
    print("***** Houshold " + str(houshold) + " dataset created ****** ")
    for week in range(len(hours_test_date)):
        #print("week "+  str(week) + " splitting started")
        result = create_features(houshold)
        features, target, features_predict, target_predict = data_split_accumlated(result, hours_test_date, week, houshold)

        clf = linear_model.Lasso(alpha=0.1, positive=True)

        clf.fit(features, target)

        print("trained on " + str(week))

        prediction_lasso = clf.predict(features_predict)

        houshold_predictions.append(prediction_lasso)

        week_timestamps = hours_test_date[week]

        target_predict['predict'] = prediction_lasso

        df.loc[week_timestamps[0]:week_timestamps[6],houshold] = target_predict['predict']

        print("=============================================")
        print(prediction_lasso)
        #print(df.loc[week_timestamps[0]:week_timestamps[6],houshold])
        print("=============================================")

    all_predictions.append(houshold_predictions)

print("--- %s seconds ---" % (time.time() - start_time))

***** Houshold 1 dataset created ****** 
[Timestamp('2019-04-29 00:00:00'), Timestamp('2019-04-29 01:00:00'), Timestamp('2019-04-29 02:00:00'), Timestamp('2019-04-29 03:00:00'), Timestamp('2019-04-29 04:00:00'), Timestamp('2019-04-29 05:00:00'), Timestamp('2019-04-29 06:00:00'), Timestamp('2019-04-29 07:00:00'), Timestamp('2019-04-29 08:00:00'), Timestamp('2019-04-29 09:00:00'), Timestamp('2019-04-29 10:00:00'), Timestamp('2019-04-29 11:00:00'), Timestamp('2019-04-29 12:00:00'), Timestamp('2019-04-29 13:00:00'), Timestamp('2019-04-29 14:00:00'), Timestamp('2019-04-29 15:00:00'), Timestamp('2019-04-29 16:00:00'), Timestamp('2019-04-29 17:00:00'), Timestamp('2019-04-29 18:00:00'), Timestamp('2019-04-29 19:00:00'), Timestamp('2019-04-29 20:00:00'), Timestamp('2019-04-29 21:00:00'), Timestamp('2019-04-29 22:00:00'), Timestamp('2019-04-29 23:00:00'), Timestamp('2019-04-30 00:00:00'), Timestamp('2019-04-30 01:00:00'), Timestamp('2019-04-30 02:00:00'), Timestamp('2019-04-30 03:00:00'), Timest

In [124]:
weeks_columns = [week for weeks in hours_test_date for week in weeks]

[Timestamp('2019-04-29 00:00:00'),
 Timestamp('2019-04-29 01:00:00'),
 Timestamp('2019-04-29 02:00:00'),
 Timestamp('2019-04-29 03:00:00'),
 Timestamp('2019-04-29 04:00:00'),
 Timestamp('2019-04-29 05:00:00'),
 Timestamp('2019-04-29 06:00:00'),
 Timestamp('2019-04-29 07:00:00'),
 Timestamp('2019-04-29 08:00:00'),
 Timestamp('2019-04-29 09:00:00'),
 Timestamp('2019-04-29 10:00:00'),
 Timestamp('2019-04-29 11:00:00'),
 Timestamp('2019-04-29 12:00:00'),
 Timestamp('2019-04-29 13:00:00'),
 Timestamp('2019-04-29 14:00:00'),
 Timestamp('2019-04-29 15:00:00'),
 Timestamp('2019-04-29 16:00:00'),
 Timestamp('2019-04-29 17:00:00'),
 Timestamp('2019-04-29 18:00:00'),
 Timestamp('2019-04-29 19:00:00'),
 Timestamp('2019-04-29 20:00:00'),
 Timestamp('2019-04-29 21:00:00'),
 Timestamp('2019-04-29 22:00:00'),
 Timestamp('2019-04-29 23:00:00'),
 Timestamp('2019-04-30 00:00:00'),
 Timestamp('2019-04-30 01:00:00'),
 Timestamp('2019-04-30 02:00:00'),
 Timestamp('2019-04-30 03:00:00'),
 Timestamp('2019-04-

In [125]:
def flatten(hous):
    return [week for weeks in hous for week in weeks]

In [126]:
flatened_predictions = []
for hous in all_predictions:
    flatened_predictions.append(flatten(hous))

In [127]:
df_from_list = pd.DataFrame([i for i in flatened_predictions], columns= [weeks_columns],index = final_index)

In [128]:
df_from_list.reset_index(inplace = True)
df_from_list = df_from_list.round(10)

In [129]:
df_from_list.to_csv("./sample_submission_HOURLY_max_eval_new_features_FILTERED_float32_ROUND_LASSO.csv", index = False)

In [130]:
df_from_list

,pseudo_id,2019-04-29 00:00:00,2019-04-29 01:00:00,2019-04-29 02:00:00,2019-04-29 03:00:00,2019-04-29 04:00:00,2019-04-29 05:00:00,2019-04-29 06:00:00,2019-04-29 07:00:00,2019-04-29 08:00:00,...,2019-09-04 14:00:00,2019-09-04 15:00:00,2019-09-04 16:00:00,2019-09-04 17:00:00,2019-09-04 18:00:00,2019-09-04 19:00:00,2019-09-04 20:00:00,2019-09-04 21:00:00,2019-09-04 22:00:00,2019-09-04 23:00:00
0,0x16cb02173ebf3059efdc97fd1819f14a2,109.989065,53.268936,45.419781,25.430671,18.526719,18.652906,13.387709,13.523009,13.757516,...,48.976109,48.976109,48.976109,48.976109,48.976109,48.976109,48.976109,48.976109,48.976109,48.976109
1,0x1c9d08cd16fce04790ef900695861e786,7.880322,5.409271,4.985641,3.560395,2.590446,2.598992,2.110154,0.957934,0.943716,...,4.150859,4.150859,4.150859,4.150859,4.150859,4.150859,4.150859,4.150859,4.150859,4.150859
2,0x1612e4cbe3b1b85c3dbcaeaa504ee8424,16.378035,5.657128,5.054165,3.762991,4.050378,3.967014,3.224080,3.126171,2.833721,...,12.530627,12.530627,12.530627,12.530627,12.530627,12.530627,12.530627,12.530627,12.530627,12.530627
3,0x20158d36236a640cf0524dba149459169,128.508238,50.494491,34.920398,31.585118,28.077995,24.128597,21.471966,18.438657,17.189437,...,64.982297,64.982297,64.982297,64.982297,64.982297,64.982297,64.982297,64.982297,64.982297,64.982297
4,0xc305005dcb1ed6128d816954c5ab9e7e,65.853397,34.604347,36.814920,19.512389,18.870784,20.202508,20.924005,13.269195,13.990244,...,28.218087,28.218087,28.218087,28.218087,28.218087,28.218087,28.218087,28.218087,28.218087,28.218087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0x12342fbadc0ca9418f2d540bb3cb8364a,2.152535,1.749078,1.728113,1.485175,1.431934,1.465033,1.391108,1.241363,1.215509,...,0.686815,0.686815,0.686815,0.686815,0.686815,0.686815,0.686815,0.686815,0.686815,0.686815
57,0x16d1816bc5d185c47de080d4c6a64bc9a,5.239516,3.926448,3.871932,3.705768,2.947199,2.475940,2.471269,2.478058,2.288622,...,2.550436,2.550436,2.550436,2.550436,2.550436,2.550436,2.550436,2.550436,2.550436,2.550436
58,0x14f480f24c435af1b8574c1c6bab38a1c,8.673817,5.814181,4.900990,4.220869,3.719921,3.451692,3.266895,2.402991,2.137812,...,3.688947,3.688947,3.688947,3.688947,3.688947,3.688947,3.688947,3.688947,3.688947,3.688947
59,0x1c7fc724d0a4f89ed1de8a0a4b302db22,2.152770,1.509494,1.257401,0.904627,0.877256,0.863553,0.883070,0.848127,0.775276,...,1.265798,1.265798,1.265798,1.265798,1.265798,1.265798,1.265798,1.265798,1.265798,1.265798


In [131]:
#h1 = pd.DataFrame(data=flatened_predictions[0],index=weeks_columns)

In [132]:
#plt.rcParams.update({'figure.figsize':(12,3), 'figure.dpi':160})
#h1.plot()
#plt.title('Energy use forecasts for houshold group' + " 1")
#plt.show()

In [133]:
import eli5
from eli5.sklearn import PermutationImportance
#feature_names = list(features.columns)
#eli5.explain_weights(model_linear, top=40, feature_names = feature_names)

In [134]:
#perm = PermutationImportance(model_linear).fit(features, target)
#eli5.show_weights(perm,top=50,feature_names = feature_names)